In [1]:
#The name of this project is 'Cryptara' and the objective of this script is to build a Robo-advisor
name = input("Welcome Crypto Investor, what is your name?")

In [2]:
#Introduction. Have user input their desired upload option. Options include importing a csv or inputting individual tickers.
if name == None:
    print('I respect your privacy, you dont mind if I call you Karen, right')
    name = "Karen"
if name != "" or name == None:
    # Store user's cash investment.
    print(f"Hello {name}, my name is Cryptara. I make your savings go to good use in the Crypto market..")
    print('')
    data_source=input("Would you like to import a csv file or input your tickers manually? Type: 'import' OR 'input'")
    print(f"{data_source} option has been selected..")
    print(f"")
    investment=input("how much cash would you like to invest?")
    investment = float(investment)
    print('')
    print(f'Once I have your list, this process starts automatically..')
    print(f'I use data from the TradingView API and filter your list, then I apply a built-in technical analysis function to determine which cryptos are good picks at this point in time')
    print(f"Then, I run hundreds of simulations to determine optimal weights for each crypto selection")
    print(f"From there, I will scan multiple exchanges to determine where you could get the most volume for the dollar at that particular time '(based on cross-exchange rates)'")
    print(f"Lastly, I will present to you an interactive Dashboard along with my future projections..")

Hello nomad, my name is Cryptara. I make your savings go to good use in the Crypto market..

import option has been selected..


Once I have your list, this process starts automatically..
I use data from the TradingView API and filter your list, then I apply a built-in technical analysis function to determine which cryptos are good picks at this point in time
Then, I run hundreds of simulations to determine optimal weights for each crypto selection
From there, I will scan multiple exchanges to determine where you could get the most volume for the dollar at that particular time '(based on cross-exchange rates)'
Lastly, I will present to you an interactive Dashboard along with my future projections..


In [3]:
#User input validation
if data_source == 'import':
    from pathlib import Path
    ticker_data = Path("csv_data/crypto_tickers1.csv")
    print(f'Thanks for using our csv file import option! Let me look through your list and see if I can validate the data..')
if data_source == 'input':
    ticker_1=input("Tell me the first ticker")
    ticker_2=input("Tell me the second ticker")
    ticker_3=input("Tell me the third ticker")
    ticker_4=input("Tell me the fourth ticker")
    ticker_5=input("Tell me the fifth ticker")
    ticker_6=input("Tell me the sixth ticker")
    ticker_7=input("Tell me the seventh ticker")
    ticker_8=input("Tell me the eigth ticker")
    ticker_9=input("Tell me the ninth ticker")
    print("Please upgrade to our Premium version of Cryptana to input more than nine tickers.")
    ticker_data = ([ticker_1,ticker_2,ticker_3,ticker_4,ticker_5,ticker_6,ticker_7,ticker_8,ticker_7,ticker_8,ticker_9])
    print(f'Got it. Give me a minute to make sure these tickers are valid')
if data_source != 'import':
    if data_source != 'input':
        data_source = input('Please try again..')

Thanks for using our csv file import option! Let me look through your list and see if I can validate the data..


In [4]:
#Import libraries and dependencies
print(f'Lets get warmed up..')
from dotenv import load_dotenv
from itertools import product
from panel.interact import interact
import datetime
import os
import random
import pandas as pd
import numpy as np
import warnings
from pathlib import Path
import panel as pn
pn.extension('plotly')
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
from plotly.offline import iplot
from plotly.subplots import make_subplots
from panel import widgets
from dotenv import load_dotenv

warnings.filterwarnings('ignore')
%matplotlib inline
exchange = 'BINANCE'

Lets get warmed up..


In [5]:
#Create TradingView variable and activate env variables
print(f'Trading View is my trusted partner, together we will see where your money should go')
from tvDatafeed import TvDatafeed,Interval
load_dotenv()
tradev_id = os.getenv("username")
tradev_secret_key = os.getenv("password")

Trading View is my trusted partner, together we will see where your money should go


In [6]:
#Log into TradingView
tv = TvDatafeed(tradev_id, tradev_secret_key, chromedriver_path=None)
print('TradingView Authentication is successfull')

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[1]/div[2]/div"}
  (Session info: headless chrome=96.0.4664.110)
, ('no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[1]/div[2]/div"}\n  (Session info: headless chrome=96.0.4664.110)', None, None)
automatic login failed
 Reinitialize tvdatafeed with auto_login=False 
you are using nologin method, data you access may be limited


TradingView Authentication is successfull


In [7]:
from pathlib import Path
ticker_data = Path("csv_data/crypto_tickers1.csv")
ticker= pd.read_csv(ticker_data, header=None)
ticker_df = pd.DataFrame(ticker)
my_crypto_list = ticker_df[0].tolist()
screener="CRYPTO"
exchange="BINANCE"
df_daily = pd.DataFrame()
print(f"Nice work! No issues have been identified with your list, let me look into these and get back to you shortly...")

Nice work! No issues have been identified with your list, let me look into these and get back to you shortly...


In [8]:
#Get historical pricing information for total population
print(f'Extracting price history..')
try:
    for ticker in my_crypto_list:
        data = tv.get_hist(
            symbol=ticker,
            exchange=exchange,
            interval=Interval.in_daily,n_bars=500)
        df_daily = df_daily.append(data)
except:
    pass
print(f'finished extracting historical prices..')

Extracting price history..


ERROR:tvDatafeed.main:Connection is already closed.


finished extracting historical prices..


In [9]:
#Grab indicator recommendations from Trading View API
print(f'Risk protocol: Filtering by analyst recommendations')
from tradingview_ta import TA_Handler, Interval, Exchange
staging_df = pd.DataFrame()
ticker_df_daily = pd.DataFrame()
for ticker in my_crypto_list:
    try:
        data = (TA_Handler(symbol=ticker,screener=screener,
                           exchange=exchange,interval=Interval.INTERVAL_1_DAY ).get_analysis().summary)
        symbol = ticker
        staging_df = list(data.values())
        final_df = (pd.DataFrame((data), index={ticker}))
        ticker_df_daily = ticker_df_daily.append(final_df)
    except:
        pass
print(f'finished extracting analyst recommendations..')

Risk protocol: Filtering by analyst recommendations
finished extracting analyst recommendations..


In [10]:
#Filter imported list by analyst recommendations
print(f'We checked what the top analysts are saying about each crypto in your list')
print(f'To reduce program speed, Cryptara will only analyze cryptos that meet a BUY or NEUTRAL classification')
initial_drop = ticker_df_daily.loc[ticker_df_daily['RECOMMENDATION']=='SELL']
initial_pass_prime = ticker_df_daily.loc[ticker_df_daily['RECOMMENDATION']=='BUY']
initial_pass_subprime = ticker_df_daily.loc[ticker_df_daily['RECOMMENDATION']=='NEUTRAL']

#initial_drop = initial_indicator['RECOMMENDATION'].str.contains('SELL')
buy_count = initial_pass_prime['RECOMMENDATION'].count() + initial_pass_subprime['RECOMMENDATION'].count()
sell_count = initial_drop['RECOMMENDATION'].count()
print('Here are the initial findings:')
print('')
print(f'It is not a good time to invest in {sell_count} of the cryptos from your list')
print(f'These {sell_count} cryptos are not in a buying position')
print(initial_drop['RECOMMENDATION'])
print(f'These {sell_count} cryptos have been dropped from consideration..')
print('')
print('')
print(f'Moving on, our initial inidicator data suggest {buy_count} cryptos from your list should be considered for further analysis:')
print('Here they are:')
print('Our Prime candidates that are in buy positions are:')
print(initial_pass_prime['RECOMMENDATION'])
print('')
print('And these are neutral for now, we can resverve for later if we need additional cryptos for analysis')
print(initial_pass_subprime['RECOMMENDATION'])

We checked what the top analysts are saying about each crypto in your list
To reduce program speed, Cryptara will only analyze cryptos that meet a BUY or NEUTRAL classification
Here are the initial findings:

It is not a good time to invest in 72 of the cryptos from your list
These 72 cryptos are not in a buying position
BTCUSD     SELL
ETHUSD     SELL
DOGEUSD    SELL
LTCUSD     SELL
VGXETH     SELL
           ... 
BUSDDAI    SELL
TROYBNB    SELL
ELFBTC     SELL
OMGETH     SELL
GLMETH     SELL
Name: RECOMMENDATION, Length: 72, dtype: object
These 72 cryptos have been dropped from consideration..


Moving on, our initial inidicator data suggest 41 cryptos from your list should be considered for further analysis:
Here they are:
Our Prime candidates that are in buy positions are:
BTCDOWNUSDT        BUY
BCHDOWNUSDT        BUY
BUSDBRL            BUY
USDTBRL            BUY
GXSBTC             BUY
MDABTC             BUY
FILDOWNUSDT        BUY
TRXDOWNUSDT        BUY
ADADOWNUSDT        BUY
PAXGU

In [11]:
#Extract index values for the tickers that are prime and subprime and format data for further analysis. 
print(f'Nice work, {name}, you found {buy_count} cryptos that might be good investments this week!')
initial_pass_prime.reset_index(inplace=True)
initial_pass_prime = initial_pass_prime['index']
initial_pass_subprime.reset_index(inplace=True)
initial_pass_subprime = initial_pass_subprime['index']
staging_df = pd.DataFrame()
staging_df1 = pd.DataFrame()
prime_df = pd.DataFrame()
subprime_df = pd.DataFrame()
print(f'Lets see what Cryptara thinks about these cryptos..')
print(f'')

Nice work, nomad, you found 41 cryptos that might be good investments this week!
Lets see what Cryptara thinks about these cryptos..



In [12]:
#Run a for loop through prime and subprime cryptos and pull in all oscilattor data for technical analysis
#Prime cryptos
print(f'Extracting oscillator data from TradingView for built in technical analysis test...')
for ticker in initial_pass_prime:
    try:
        data = (TA_Handler(symbol=ticker,screener=screener,
                           exchange=exchange,interval=Interval.INTERVAL_1_DAY ).get_analysis().indicators)
        symbol = ticker
        staging_df = list(data.values())
        final_df = (pd.DataFrame((data), index={ticker}))
        prime_df = prime_df.append(final_df)
    except:
        pass

#Sub-prime cryptos
for ticker in initial_pass_subprime:
    try:
        data = (TA_Handler(symbol=ticker,screener=screener,
                           exchange=exchange,interval=Interval.INTERVAL_1_DAY ).get_analysis().indicators)
        symbol = ticker
        staging_df1 = list(data.values())
        final_df = (pd.DataFrame((data), index={ticker}))
        subprime_df = subprime_df.append(final_df)
    except:
        pass
print('Extracted oscillator data, initializing technical analysis test on key trend indicators..')
prime_df.head(3)

Extracting oscillator data from TradingView for built in technical analysis test...
Extracted oscillator data, initializing technical analysis test on key trend indicators..


,Recommend.Other,Recommend.All,Recommend.MA,RSI,RSI[1],Stoch.K,Stoch.D,Stoch.K[1],Stoch.D[1],CCI20,...,Pivot.M.Demark.R1,open,P.SAR,BB.lower,BB.upper,AO[2],volume,change,low,high
BTCDOWNUSDT,0.0,0.200000,0.400000,60.081906,63.310083,62.112227,52.731661,49.903076,48.727946,95.837269,...,0.011857,0.01351,0.011300,0.008423,0.014084,0.002145,2.533834e+08,-3.822460,0.012667,0.013905
BCHDOWNUSDT,0.0,0.133333,0.266667,53.015949,56.213846,41.544162,29.416686,24.160609,23.902927,85.488652,...,0.223500,0.22740,0.284003,0.160506,0.231514,0.007035,6.591781e+05,-5.621432,0.210600,0.234900
BUSDBRL,0.0,0.400000,0.800000,53.359437,51.985588,53.593564,50.425228,47.792494,48.822664,14.872486,...,5.720000,5.69700,5.574160,5.618780,5.774320,0.037535,5.153470e+06,0.175623,5.680000,5.707000


In [13]:
#Extract oscillator information for all cryptos that meet the 'buy threshold'  
crypataras_picks = []
print(f'Determining which cryptos are in an optimal buying position based on trend indicators...')
print('Relative Strength Index (RSI), Moving Average Covergence and Divergence (MACD), Stachator positioning and the Awesome Oscillators')
print(f'By doing this, we can see which cryptos are currently overbought or oversold based on historical prices..')
print(f'')
def strong_performers():
    for i, j in prime_df.iterrows():
        if j['RSI'] > 45 and j['RSI'] < 90:
            print(f'{i} has passed the rsi_test, moving to moving average covergence and divergence analysis..')
            if j['MACD.macd'] > 0 and j['MACD.macd'] < 40:
                print(f'{i} has passed our moving average covergence and divergence analysis, moving to stochator analysis..')
                if j['Stoch.K'] > 35 and j['Stoch.K'] < 85:
                    if j['Stoch.D'] > 40 and j['Stoch.D'] < 85:
                        if j['Stoch.RSI.K'] > 30 and j['Stoch.RSI.K'] < 80:
                            print(f'{i} has passed our multiple stochator analysis, testing the Awesome Oscillators..')
                            if j['AO'] > 0 and j['AO'] < 55:
                                if i not in crypataras_picks:
                                    print(f'Cryptara has selected {i} ')
                                    crypataras_picks.append(i)
                                
    else:
        print(f'FAIL! dropping crypto from further consideration..')

strong_performers()
initial_count = len(crypataras_picks)
print(f'')
print('Technical Analysis test is complete.')
print('')
print('Cryptara has analyzed key technical indicators and suggest that the following crypto assets')
print('are in optimal buying positions')
print(f'Cryptara chooses: {crypataras_picks}')

Determining which cryptos are in an optimal buying position based on trend indicators...
Relative Strength Index (RSI), Moving Average Covergence and Divergence (MACD), Stachator positioning and the Awesome Oscillators
By doing this, we can see which cryptos are currently overbought or oversold based on historical prices..

BTCDOWNUSDT has passed the rsi_test, moving to moving average covergence and divergence analysis..
BTCDOWNUSDT has passed our moving average covergence and divergence analysis, moving to stochator analysis..
BTCDOWNUSDT has passed our multiple stochator analysis, testing the Awesome Oscillators..
Cryptara has selected BTCDOWNUSDT 
BCHDOWNUSDT has passed the rsi_test, moving to moving average covergence and divergence analysis..
BCHDOWNUSDT has passed our moving average covergence and divergence analysis, moving to stochator analysis..
BUSDBRL has passed the rsi_test, moving to moving average covergence and divergence analysis..
BUSDBRL has passed our moving average 

In [14]:
print(f'{initial_count} tickers are in optimal buying positions..testing data population requirements..')
print(f'')
#Test if we have a minimum of five tickers for final analyses.
if len(crypataras_picks) > 5:
        crypataras_picks.remove(random.choice(crypataras_picks))
        print(f'removed ticker from list to meet maxiumum requirements')

if len(crypataras_picks) < 5:
    print('This program requires a minimum of five cryptos to run remaining analysis.')
    computer_selection_1 = subprime_df.sample().reset_index()
    ticker_random_1 = computer_selection_1.iloc[:, 0].tolist()
    string=''.join([str(item) for item in ticker_random_1])
    if string in crypataras_picks:
        pass
    else:
        crypataras_picks.append(string)
        print(f'added {string} from BUY list to meet minimum requirements')
        
if len(crypataras_picks) < 5:
    print('This program requires a minimum of five cryptos to run remaining analysis.')
    computer_selection_2 = subprime_df.sample().reset_index()
    ticker_random_2 = computer_selection_2.iloc[:, 0].tolist()
    string=''.join([str(item) for item in ticker_random_2])
    if string in crypataras_picks:
        pass
    else:
        crypataras_picks.append(string)
        print(f'added {string} from BUY list to meet minimum requirements')
        
if len(crypataras_picks) < 5:
    print('This program requires a minimum of five cryptos to run remaining analysis')
    computer_selection_3 = subprime_df.sample().reset_index()
    ticker_random_3 = computer_selection_3.iloc[:, 0].tolist()
    string=''.join([str(item) for item in ticker_random_3])
    if string in crypataras_picks:
        pass
    else:
        crypataras_picks.append(string)
        print(f'added {string} from BUY list to meet minimum requirements')
        
if len(crypataras_picks) < 5:
    print('This program requires a minimum of five cryptos to run remaining analysis')
    computer_selection_4 = subprime_df.sample().reset_index()
    ticker_random_4 = computer_selection_4.iloc[:, 0].tolist()
    string=''.join([str(item) for item in ticker_random_4])
    if string in crypataras_picks:
        pass
    else:
        crypataras_picks.append(string)
        print(f'added {string} from BUY list to meet minimum requirements')

if len(crypataras_picks) > 5:
        crypataras_picks.remove(random.choice(crypataras_picks))
        print(f'removed ticker from list to meet maxiumum requirements')
if len(crypataras_picks) > 5:
        crypataras_picks.remove(random.choice(crypataras_picks))
        print(f'removed ticker from list to meet maxiumum requirements')
if len(crypataras_picks) > 5:
        crypataras_picks.remove(random.choice(crypataras_picks))
        print(f'removed ticker from list to meet maxiumum requirements')

print(f'')
print(f'Here are the final Cryptara picks:')
print(f'')
csv_export = pd.DataFrame(crypataras_picks)
csv_export.to_csv('csv_data/cryptaras_final_cryptos.csv')
crypataras_picks

6 tickers are in optimal buying positions..testing data population requirements..

removed ticker from list to meet maxiumum requirements

Here are the final Cryptara picks:



['BTCDOWNUSDT', 'USDTBRL', 'MDABTC', 'FILDOWNUSDT', 'PAXGBTC']

In [15]:
from tvDatafeed import TvDatafeed,Interval
load_dotenv()
tradev_id = os.getenv("username")
tradev_secret_key = os.getenv("password")
print(f'Gathering return data on Cryptaras picks..')
tv = TvDatafeed(tradev_id, tradev_secret_key, chromedriver_path=None)

Gathering return data on Cryptaras picks..


ERROR:tvDatafeed.main:Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[1]/div[2]/div"}
  (Session info: headless chrome=96.0.4664.110)
, ('no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="overlap-manager-root"]/div/span/div[1]/div/div/div[1]/div[2]/div"}\n  (Session info: headless chrome=96.0.4664.110)', None, None)
ERROR:tvDatafeed.main:automatic login failed
 Reinitialize tvdatafeed with auto_login=False 


In [16]:
print(f'Extracting most recent prices..')
final_crypto_df = pd.DataFrame()
bars= 1000
for all_picks in crypataras_picks:
    picks_df = tv.get_hist(symbol=all_picks,
                       exchange=exchange,
                       interval=Interval.in_daily,
                       n_bars=bars)
    final_crypto_df = final_crypto_df.append(picks_df)
print(f'Give me a moment while I reformat some datasets to obtain an optimal performance strategy..')

Extracting most recent prices..
Give me a moment while I reformat some datasets to obtain an optimal performance strategy..


In [17]:
#Format dataframe.
#Step 1: extract exchange and ticker from 'symbol' column into separate columns.
final_df_cleaner = final_crypto_df
final_df_cleaner[['exchange', 'ticker']] = final_df_cleaner['symbol'].str.split(':',expand=True)
#Step 2: Extract ticker and close values, reset index
final_df_cleaner = final_df_cleaner[['ticker','close']]
final_df_cleaner = final_df_cleaner[['ticker','close']].reset_index()
#Step3: Pivot data to make ticker values the column headers
final_df_cleaner = final_df_cleaner.pivot(index='datetime',
           columns='ticker',
           values='close')
#Step4: calculate return data
print('Extracted the historical returns for Cryparas picks..')
crypto_pick_returns = final_df_cleaner.pct_change().dropna()
crypto_pick_returns.tail(5)

Extracted the historical returns for Cryparas picks..


ticker,BTCDOWNUSDT,FILDOWNUSDT,MDABTC,PAXGBTC,USDTBRL
datetime,,,,,
2021-12-13 18:00:00,-0.078949,-0.056893,0.012019,-0.041525,-0.002443
2021-12-14 18:00:00,-0.030429,-0.087538,-0.018211,-0.007084,-0.000525
2021-12-15 18:00:00,0.068942,0.063473,0.016935,0.037047,0.004551
2021-12-16 18:00:00,0.081742,0.080779,0.022998,0.033607,-0.007144
2021-12-17 18:00:00,-0.039080,-0.051320,-0.017054,-0.014593,0.001580


In [18]:
#Step5: Calculate Cumulative Returns
print('Getting a feel for each cryptos LTD cumulative returns ..')
cumulative_daily_returns = (1 + crypto_pick_returns).cumprod() -1
cumulative_daily_returns.tail(10)

Getting a feel for each cryptos LTD cumulative returns ..


ticker,BTCDOWNUSDT,FILDOWNUSDT,MDABTC,PAXGBTC,USDTBRL
datetime,,,,,
2021-12-08 18:00:00,-0.997177,-0.999911,-0.600196,-0.768323,-0.003875
2021-12-09 18:00:00,-0.997120,-0.999920,-0.579993,-0.765093,-0.000528
2021-12-10 18:00:00,-0.997423,-0.999922,-0.583578,-0.775528,-0.002290
2021-12-11 18:00:00,-0.997527,-0.999920,-0.596611,-0.778385,-0.002290
2021-12-12 18:00:00,-0.997070,-0.999901,-0.593353,-0.762174,0.009336
2021-12-13 18:00:00,-0.997301,-0.999906,-0.588465,-0.772050,0.006870
2021-12-14 18:00:00,-0.997383,-0.999915,-0.595960,-0.773665,0.006341
2021-12-15 18:00:00,-0.997203,-0.999909,-0.589117,-0.765280,0.010921
2021-12-16 18:00:00,-0.996974,-0.999902,-0.579668,-0.757391,0.003699


In [19]:
#Step5: Calculate Cumulative Returns
print('Activating Smartfolio program to determine optimal investment strategy..')
print('')
print('Logging returns to normalize dataset..')
log_returns= np.log(abs(cumulative_daily_returns))
log_returns.tail()

Activating Smartfolio program to determine optimal investment strategy..

Logging returns to normalize dataset..


ticker,BTCDOWNUSDT,FILDOWNUSDT,MDABTC,PAXGBTC,USDTBRL
datetime,,,,,
2021-12-13 18:00:00,-0.002702,-0.000094,-0.530237,-0.258706,-4.980617
2021-12-14 18:00:00,-0.002620,-0.000085,-0.517582,-0.256617,-5.060659
2021-12-15 18:00:00,-0.002801,-0.000091,-0.529131,-0.267514,-4.517044
2021-12-16 18:00:00,-0.003030,-0.000098,-0.545300,-0.277875,-5.599656
2021-12-17 18:00:00,-0.002912,-0.000093,-0.533010,-0.273212,-5.242981


In [20]:
#Calculate volatility and Initialize variables 
print(f'Running multiple scenaries to determine appropriate weights for selected cryptos')

num_of_portfolios= 5000
weight= np.zeros((num_of_portfolios,len(crypataras_picks)))
expected_return=  np.zeros(num_of_portfolios)
expected_volatility= np.zeros(num_of_portfolios)
sharpe_ratio= np.zeros(num_of_portfolios)
expected_return
mean_log_ret= log_returns.mean()
sigma = log_returns.cov()

for i in range (num_of_portfolios):
    # generate random weight vectos 
    w= np.array(np.random.random(len(crypataras_picks)))
        #sum of weights equal to 1 
    w= w/ np.sum(w)
    weight[i,:]= w
    
    # expected log return 
    expected_return[i]= np.sum(mean_log_ret*w)
    
    #expected volatility 
    expected_volatility[i]= np.sqrt(np.dot(w.T,np.dot(sigma,w)))
    
    # sharpe ratio
    sharpe_ratio[i] =  expected_return[i]/expected_volatility[i]
    
print(f'Here are the weights: ')
# Weighted portfolio that sharpe ratio
max_index = sharpe_ratio.argmax()
weight[max_index,:]

Running multiple scenaries to determine appropriate weights for selected cryptos
Here are the weights: 


array([0.41508222, 0.49481258, 0.027136  , 0.04410639, 0.01886281])

In [21]:
# Portfolio weights are exported to a csv that can be used for monteclaro simulation
print(f'Loading Smartfolio weights..')
print(f'')
print(f'')
portfolio_weights = np.array([weight[max_index,:]])
portfolio_weights = portfolio_weights.tolist() 
smartfolio_weights = pd.DataFrame(portfolio_weights)
total_invested=(smartfolio_weights*investment).round(2)
smartfolio_weights.to_csv('csv_data/smartfolio_weights.csv')
smartfolio_weights_df = smartfolio_weights
smartfolio_weights_df.columns = ([crypataras_picks])
print(f' ............and DONE!')

print('')
print('')
print(f'At this moment in time, here is how you should allocate your {investment} investment...') 
print('')
#print(f' The optimal portfolio weight for '
print(f'With a {smartfolio_weights.iloc[0][0]:.3f} weight, Cryptara suggests that ${total_invested.iloc[0][0]} should be allocated to {crypataras_picks[0]}.')
print('')
print(f'With a {smartfolio_weights.iloc[0][1]:.3f} weight, Cryptara suggests that ${total_invested.iloc[0][1]} should be allocated to {crypataras_picks[1]}.')
print('')
print(f'With a {smartfolio_weights.iloc[0][2]:.3f} weight, Cryptara suggests that ${total_invested.iloc[0][2]} should be allocated to {crypataras_picks[2]}.')
print('')
print(f'With a {smartfolio_weights.iloc[0][3]:.3f} weight, Cryptara suggests that ${total_invested.iloc[0][3]} should be allocated to {crypataras_picks[3]}.')
print('')
print(f'With a {smartfolio_weights.iloc[0][4]:.3f} weight, Cryptara suggests that ${total_invested.iloc[0][4]} should be allocated to {crypataras_picks[4]}.')

Loading Smartfolio weights..


 ............and DONE!


At this moment in time, here is how you should allocate your 3516351351.0 investment...

With a 0.415 weight, Cryptara suggests that $1459574916.13 should be allocated to BTCDOWNUSDT.

With a 0.495 weight, Cryptara suggests that $1739934881.83 should be allocated to USDTBRL.

With a 0.027 weight, Cryptara suggests that $95419708.21 should be allocated to MDABTC.

With a 0.044 weight, Cryptara suggests that $155093565.54 should be allocated to FILDOWNUSDT.

With a 0.019 weight, Cryptara suggests that $66328279.29 should be allocated to PAXGBTC.


In [22]:
import itertools
from itertools import product
from MCForecastTools import MCSimulation
%matplotlib inline

In [23]:
mcsims_df = pd.DataFrame()
for final_cryptos in crypataras_picks:
    mcsims = tv.get_hist(symbol=final_cryptos,
                       exchange=exchange,
                       interval=Interval.in_daily,
                       n_bars=bars)
    mcsims_df = mcsims_df.append(mcsims)

ticker1_df = mcsims_df.loc[mcsims_df.symbol==f'{exchange}' + ':' +f'{crypataras_picks[0]}']
ticker2_df = mcsims_df.loc[mcsims_df.symbol==f'{exchange}' + ':' +f'{crypataras_picks[1]}']
ticker3_df = mcsims_df.loc[mcsims_df.symbol==f'{exchange}' + ':' +f'{crypataras_picks[2]}']
ticker4_df = mcsims_df.loc[mcsims_df.symbol==f'{exchange}' + ':' +f'{crypataras_picks[3]}']
ticker5_df = mcsims_df.loc[mcsims_df.symbol==f'{exchange}' + ':' +f'{crypataras_picks[4]}']
sim_data = pd.concat([ticker1_df, ticker2_df, ticker3_df, ticker4_df, ticker5_df], axis='columns', join='inner')


print(f'Finished extracting extracting historical prices for selected cryptos')
mcsims_df.head(5)

Finished extracting extracting historical prices for selected cryptos


,symbol,open,high,low,close,volume
datetime,,,,,,
2020-05-13 19:00:00,BINANCE:BTCDOWNUSDT,10.15,10.92,9.41,9.79,7767.94
2020-05-14 19:00:00,BINANCE:BTCDOWNUSDT,9.85,11.11,9.80,10.65,12910.60
2020-05-15 19:00:00,BINANCE:BTCDOWNUSDT,10.65,10.99,10.20,10.62,31347.33
2020-05-16 19:00:00,BINANCE:BTCDOWNUSDT,10.69,10.70,9.57,9.96,68080.72
2020-05-17 19:00:00,BINANCE:BTCDOWNUSDT,9.90,10.35,9.43,9.81,257225.85


In [24]:
print(f'Now that we know exactly how to pool your funds, Monte Carlo should help us predict where this {investment} should go in the future...')

mdex = itertools.product(crypataras_picks, ticker1_df.columns)
columns = []
for x in mdex:
 columns.append(x)
sim_data.columns = pd.MultiIndex.from_tuples(columns) 

Now that we know exactly how to pool your funds, Monte Carlo should help us predict where this 3516351351.0 should go in the future...


In [25]:
print('')
print(f'Initializing Monte Carlo Simulation..')
print(f'')
MC_even_dist = MCSimulation(
    portfolio_data = sim_data, 
    weights = [smartfolio_weights.iloc[0][0], smartfolio_weights.iloc[0][1], smartfolio_weights.iloc[0][2], smartfolio_weights.iloc[0][3], smartfolio_weights.iloc[0][4]],
    num_simulation=100,
    num_trading_days = 126
)
cumula_return = MC_even_dist.calc_cumulative_return()
cumula_return.to_csv('csv_data/mc_df.csv')


Initializing Monte Carlo Simulation..

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.


In [26]:
cumula_return = MC_even_dist.calc_cumulative_return()
simulated_returns_data = {
    "mean": list(MC_even_dist.simulated_return.mean(axis=1)),
    "median": list(MC_even_dist.simulated_return.median(axis=1)),
    "min": list(MC_even_dist.simulated_return.min(axis=1)),
    "max": list(MC_even_dist.simulated_return.max(axis=1))
}
df_simulated_returns = pd.DataFrame(simulated_returns_data)
df_simulated_returns.head()
df_simulated_returns.to_csv('csv_data/mc_stats.csv')
crypto_pick_returns.head()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.


ticker,BTCDOWNUSDT,FILDOWNUSDT,MDABTC,PAXGBTC,USDTBRL
datetime,,,,,
2020-10-20 19:00:00,-0.144267,0.182166,0.003584,-0.062422,-0.002995
2020-10-21 19:00:00,-0.021500,0.271429,0.270779,-0.025306,-0.005300
2020-10-22 19:00:00,0.004823,-0.327341,0.101686,0.000612,0.007993
2020-10-23 19:00:00,-0.032800,-0.171269,-0.056122,-0.005842,-0.000176
2020-10-24 19:00:00,0.017370,-0.642838,0.009828,-0.000273,0.000352


In [27]:
#Extract data for visualization models. Reformat datetime stamp to be in short-date format (MONTH/DAY/YEAR) and reformat volume column from exponential format
#Entire Crypto Selections
my_crypto_portfolio = crypataras_picks
dashboard_df = pd.DataFrame()
for ticker in my_crypto_portfolio:
    data = tv.get_hist(
        symbol=ticker,
        exchange=exchange,
        interval=Interval.in_daily,n_bars=1000)
    dashboard_df = dashboard_df.append(data)

dashboard_df.index = pd.to_datetime(dashboard_df.index, errors='coerce').date

dashboard_df['volume'] = (dashboard_df['volume'].astype(float)/1000).round(4).astype(float)

dashboard_df.to_csv('csv_data/daily_ohlc.csv')

In [28]:
#Format datasets for Visualization script.
#Cryto one visualization data
df_1_crypto = pd.DataFrame()
data_1 = tv.get_hist(symbol=crypataras_picks[0],exchange=exchange,interval=Interval.in_daily,n_bars=1000)
df_1_crypto = df_1_crypto.append(data_1)
df_1_crypto.index = pd.to_datetime(df_1_crypto.index, errors='coerce').date
df_1_crypto['volume'] = (df_1_crypto['volume'].astype(float)/1000).round(4).astype(float)
df_1_crypto.to_csv('csv_data/ticker_1.csv')
#Cryto two visualization data
df_2_crypto = pd.DataFrame()
data_2 = tv.get_hist(symbol=crypataras_picks[1],exchange=exchange,interval=Interval.in_daily,n_bars=1000)
df_2_crypto = df_2_crypto.append(data_2)
df_2_crypto.index = pd.to_datetime(df_2_crypto.index, errors='coerce').date
df_2_crypto['volume'] = (df_2_crypto['volume'].astype(float)/1000).round(4).astype(float)
df_2_crypto.to_csv('csv_data/ticker_2.csv')
#Cryto three visualization data
df_3_crypto = pd.DataFrame()
data_3 = tv.get_hist(symbol=crypataras_picks[2],exchange=exchange,interval=Interval.in_daily,n_bars=1000)
df_3_crypto = df_3_crypto.append(data_3)
df_3_crypto.index = pd.to_datetime(df_3_crypto.index, errors='coerce').date
df_3_crypto['volume'] = (df_3_crypto['volume'].astype(float)/1000).round(4).astype(float)
df_3_crypto.to_csv('csv_data/ticker_3.csv')
#Cryto four visualization data
df_4_crypto = pd.DataFrame()
data_4 = tv.get_hist(symbol=crypataras_picks[3],exchange=exchange,interval=Interval.in_daily,n_bars=1000)
df_4_crypto = df_4_crypto.append(data_4)
df_4_crypto.index = pd.to_datetime(df_4_crypto.index, errors='coerce').date
df_4_crypto['volume'] = (df_4_crypto['volume'].astype(float)/1000).round(4).astype(float)
df_4_crypto.to_csv('csv_data/ticker_4.csv')
#Cryto five visualization data
df_5_crypto = pd.DataFrame()
data_5 = tv.get_hist(symbol=crypataras_picks[4],exchange=exchange,interval=Interval.in_daily,n_bars=1000)
df_5_crypto = df_5_crypto.append(data_5)
df_5_crypto.index = pd.to_datetime(df_5_crypto.index, errors='coerce').date
df_5_crypto['volume'] = (df_5_crypto['volume'].astype(float)/1000).round(4).astype(float)
df_5_crypto.to_csv('csv_data/ticker_5.csv')
print(f'Cryptara has finished selecting the best cryptos and determining the optimal weights')

print('DashBoard Visualizations:')

mcs_path = Path("csv_data/mc_df.csv")
mc_df = pd.read_csv(mcs_path, index_col=[0])
mc_df.index.name="Days"
mc_path = Path("csv_data/mc_stats.csv")
df_simulated_returns = pd.read_csv(mc_path, index_col=[0])
df_simulated_returns.index.name="Days"
df_simulated_returns.head(5)


print(f"You've chosen to invest {investment} in a diverse cryptocurrency portfolio!")
print("This simulation can help you visualize the kind of returns your initial investment might produce.")

# Set initial investment
initial_investment = investment

# Multiply an initial investment by the daily returns of simulative stock prices to return the progression of daily returns in terms of money
cumulative_pnl = initial_investment * df_simulated_returns

# Display sample data
cumulative_pnl.head(5)

Cryptara has finished selecting the best cryptos and determining the optimal weights
DashBoard Visualizations:
You've chosen to invest 3516351351.0 in a diverse cryptocurrency portfolio!
This simulation can help you visualize the kind of returns your initial investment might produce.


,mean,median,min,max
Days,,,,
0,3.516351e+09,3.516351e+09,3.516351e+09,3.516351e+09
1,3.493691e+09,3.495030e+09,3.060969e+09,3.839367e+09
2,3.472836e+09,3.466139e+09,3.028395e+09,3.984511e+09
3,3.450089e+09,3.429078e+09,2.929964e+09,4.010728e+09
4,3.426834e+09,3.421502e+09,2.847271e+09,4.047708e+09


In [29]:
print("When current price falls below the 'lower-bb' we can expect prices to begin to rise - this indicates a good BUY opportunity.") 
print("When current price raises above the 'upper-bb' we can expect prices to begin to fall soon - this presents an opportunity to SELL.")
#read in COINBASE ticker dataframe
df_daily = pd.read_csv('csv_data/daily_ohlc.csv', index_col=[0])
crypataras_picks = pd.read_csv('csv_data/cryptaras_final_cryptos.csv', index_col=[0])
crypataras_picks = crypataras_picks.values.tolist()

ticker_1_df = pd.read_csv('csv_data/ticker_1.csv', index_col=[0])
ticker_2_df = pd.read_csv('csv_data/ticker_2.csv', index_col=[0])
ticker_3_df = pd.read_csv('csv_data/ticker_3.csv', index_col=[0])
ticker_4_df = pd.read_csv('csv_data/ticker_4.csv', index_col=[0])
ticker_5_df = pd.read_csv('csv_data/ticker_5.csv', index_col=[0])
df_daily.index.name = "Date"

ticker_1_df.index.name = "Date" 
ticker_2_df.index.name = "Date"
ticker_3_df.index.name = "Date"
ticker_4_df.index.name = "Date"
ticker_5_df.index.name = "Date"

#round volume to 2 decimals for plots
ticker_1_df['volume']=ticker_1_df['volume'].round(2)
ticker_2_df['volume']=ticker_2_df['volume'].round(2)
ticker_3_df['volume']=ticker_3_df['volume'].round(2)
ticker_4_df['volume']=ticker_4_df['volume'].round(2)
ticker_5_df['volume']=ticker_5_df['volume'].round(2)

#.loc COINBASE tickers Symbol, Open, High, Low, Close, Volume data as new df
ticker_1_df=ticker_1_df.tail(60).reset_index()
ticker_2_df=ticker_2_df.tail(60).reset_index()
ticker_3_df=ticker_3_df.tail(60).reset_index()
ticker_4_df=ticker_4_df.tail(60).reset_index()
ticker_5_df=ticker_5_df.tail(60).reset_index()


# dashboard components

# mcsftool plot simulation (120)
fig1 = px.line(mc_df)
fig1 = fig1.update_layout(
    xaxis_title="Predicted Time Period(Days)",
    yaxis_title="Daily Return",
    showlegend=False,
    title='Simulation of Returns on Portfolio Investment')
fig1=fig1.update_layout(
    height=500,
    width=1100)

When current price falls below the 'lower-bb' we can expect prices to begin to rise - this indicates a good BUY opportunity.
When current price raises above the 'upper-bb' we can expect prices to begin to fall soon - this presents an opportunity to SELL.


In [30]:
#stat plot 
fig2 = go.Figure()
fig2 = fig2.add_trace(go.Scatter(x=df_simulated_returns.index, y=df_simulated_returns['mean'],
                    mode='lines',
                    name='mean returns',
                    line=dict(color="salmon", width=1),))
fig2 =fig2.add_trace(go.Scatter(x=df_simulated_returns.index, y=df_simulated_returns['median'],
                    mode='lines',
                    name='median returns',
                    line=dict(color="teal", width=1),))
fig2 =fig2.add_trace(go.Scatter(x=df_simulated_returns.index, y=df_simulated_returns['min'],
                    mode='lines',
                    name='minimum returns',
                    line=dict(color="darkred", width=1),))
fig2 =fig2.add_trace(go.Scatter(x=df_simulated_returns.index, y=df_simulated_returns['max'],
                    mode='lines',
                    name='maximum returns',
                    line=dict(color="olive", width=1),))
fig2 =fig2.update_layout(
    height=500,
    width=1200,
    xaxis_title="Number of Days(in Simulation)",
    yaxis_title="Daily Returns",
    legend_title="Stats",
    title="Statistical Analysis for Simulation of Return on Investment")

In [31]:
#Plot simulated investment portfolio returns with initial investment number (values are in USD)
fig3 = go.Figure()
fig3 =fig3.add_trace(go.Scatter(x=cumulative_pnl.index, y=cumulative_pnl['mean'],
                    mode='lines',
                    name='mean returns',
                    line=dict(color="salmon", width=1),))
fig3 =fig3.add_trace(go.Scatter(x=cumulative_pnl.index, y=cumulative_pnl['median'],
                    mode='lines',
                    name='median returns',
                    line=dict(color="teal", width=1),))
fig3 =fig3.add_trace(go.Scatter(x=cumulative_pnl.index, y=cumulative_pnl['min'],
                    mode='lines',
                    name='minimum returns',
                    line=dict(color="darkred", width=1),))
fig3 =fig3.add_trace(go.Scatter(x=cumulative_pnl.index, y=cumulative_pnl['max'],
                    mode='lines',
                    name='maximum returns',
                    line=dict(color="olive", width=1),))
fig3 =fig3.update_layout(
    height=500,
    width=1200,
    xaxis_title="Time Period(Days)",
    yaxis_title="Returns(USD)",
    legend_title="Stats",
    title=f"{initial_investment} Initial Investment - Predicted Earnings")

In [32]:
# calculate sma and std for upper and lower bb for each df in df_list
df_list = [ticker_1_df, ticker_2_df, ticker_3_df, ticker_4_df, ticker_5_df]
WINDOW=30
for df in df_list:
    df['sma'] = df['close'].rolling(WINDOW).mean()
    df['std'] = df['close'].rolling(WINDOW).std(ddof = 0)

In [33]:
# RSI PLOT 1
# Create subplots with 2 rows; top for candlestick price, and bottom for bar volume
rsi_plot1 = make_subplots(rows = 2, cols = 1, shared_xaxes = True, subplot_titles = (f'{ticker_1_df.symbol[0]} Relative Strength Index', None), vertical_spacing = 0.2, row_heights = [1.2, .8])

# Candlestick Plot
rsi_plot1=rsi_plot1.add_trace(go.Candlestick(x = ticker_1_df['Date'],
                             open = ticker_1_df['open'],
                             high = ticker_1_df['high'],
                             low = ticker_1_df['low'],
                             close = ticker_1_df['close'], showlegend=False,
                             increasing_line_color= 'olive', decreasing_line_color= 'darkred',
                             name = 'candlestick'),
              row = 1, col = 1)

# Moving Average
rsi_plot1=rsi_plot1.add_trace(go.Scatter(x = ticker_1_df['Date'],
                         y = ticker_1_df['sma'],
                         line_color = 'black',
                         name = 'sma'),
              row = 1, col = 1)

# Upper Bound
rsi_plot1=rsi_plot1.add_trace(go.Scatter(x = ticker_1_df['Date'],
                         y = ticker_1_df['sma'] + (ticker_1_df['std'] * 2),
                         line_color = 'tan',
                         line = {'dash': 'dash'},
                         name = 'upper band',
                         opacity = 0.5),
              row = 1, col = 1) 
# Lower Bound fill in between with parameter 'fill': 'tonexty'
rsi_plot1=rsi_plot1.add_trace(go.Scatter(x = ticker_1_df['Date'],
                         y = ticker_1_df['sma'] - (ticker_1_df['std'] * 2),
                         line_color = 'tan',
                         line = {'dash': 'dash'},
                         fill = 'tonexty',
                         name = 'lower band',
                         opacity = 0.5),
              row = 1, col = 1)

# Labels & Titles
rsi_plot1=rsi_plot1.update_layout(
    height=500,
    width=1200,
    xaxis_title="Volume(Millions)",
    yaxis_title="Price",
    legend_title="Indicators",
    )
# ----------------

# Volume Plot
rsi_plot1=rsi_plot1.add_trace(go.Bar(x = df['Date'], y = df['volume'], showlegend=False), 
          row = 2, col = 1)
# Remove range slider; (short time frame)
rsi_plot1=rsi_plot1.update(layout_xaxis_rangeslider_visible=True)
#rsi_plot1.show()

In [34]:
#RSI PLOT 2 
# Create subplots with 2 rows; top for candlestick price, and bottom for bar volume
rsi_plot2 = make_subplots(rows = 2, cols = 1, shared_xaxes = True, subplot_titles = (f'{ticker_2_df.symbol[0]} Relative Strength Index', None), vertical_spacing = 0.2, row_heights = [1.2, .8])

# Candlestick Plot
rsi_plot2=rsi_plot2.add_trace(go.Candlestick(x = ticker_2_df['Date'],
                             open = ticker_2_df['open'],
                             high = ticker_2_df['high'],
                             low = ticker_2_df['low'],
                             close = ticker_2_df['close'], showlegend=False,
                             increasing_line_color= 'olive', decreasing_line_color= 'darkred',
                             name = 'candlestick'),
              row = 1, col = 1)

# Moving Average
rsi_plot2=rsi_plot2.add_trace(go.Scatter(x = ticker_2_df['Date'],
                         y = ticker_2_df['sma'],
                         line_color = 'black',
                         name = 'sma'),
              row = 1, col = 1)

# Upper Bound
rsi_plot2=rsi_plot2.add_trace(go.Scatter(x = ticker_2_df['Date'],
                         y = ticker_2_df['sma'] + (ticker_2_df['std'] * 2),
                         line_color = 'tan',
                         line = {'dash': 'dash'},
                         name = 'upper band',
                         opacity = 0.5),
              row = 1, col = 1) 
# Lower Bound fill in between with parameter 'fill': 'tonexty'
rsi_plot2=rsi_plot2.add_trace(go.Scatter(x = ticker_2_df['Date'],
                         y = ticker_2_df['sma'] - (ticker_2_df['std'] * 2),
                         line_color = 'tan',
                         line = {'dash': 'dash'},
                         fill = 'tonexty',
                         name = 'lower band',
                         opacity = 0.5),
              row = 1, col = 1)

# Labels & Titles
rsi_plot2=rsi_plot2.update_layout(
    height=500,
    width=1200,
    xaxis_title="Volume(Millions)",
    yaxis_title="Price",
    legend_title="Indicators",
    )
# ----------------

# Volume Plot
rsi_plot2=rsi_plot2.add_trace(go.Bar(x = ticker_2_df['Date'], y = ticker_2_df['volume'], showlegend=False), 
          row = 2, col = 1)
# Remove range slider; (short time frame)
rsi_plot2=rsi_plot2.update(layout_xaxis_rangeslider_visible=True)
#rsi_plot2.show()

In [35]:
#RSI PLOT 3 ticker_3_df
# Create subplots with 2 rows; top for candlestick price, and bottom for bar volume
rsi_plot3 = make_subplots(rows = 2, cols = 1, shared_xaxes = True, subplot_titles = (f'{ticker_3_df.symbol[0]} Relative Strength Index', None), vertical_spacing = 0.2, row_heights = [1.2, .8])

# Candlestick Plot
rsi_plot3=rsi_plot3.add_trace(go.Candlestick(x = ticker_3_df['Date'],
                             open = ticker_3_df['open'],
                             high = ticker_3_df['high'],
                             low = ticker_3_df['low'],
                             close = ticker_3_df['close'], showlegend=False,
                             increasing_line_color= 'olive', decreasing_line_color= 'darkred',
                             name = 'candlestick'),
              row = 1, col = 1)

# Moving Average
rsi_plot3=rsi_plot3.add_trace(go.Scatter(x = ticker_3_df['Date'],
                         y = ticker_3_df['sma'],
                         line_color = 'black',
                         name = 'sma'),
              row = 1, col = 1)

# Upper Bound
rsi_plot3=rsi_plot3.add_trace(go.Scatter(x = ticker_3_df['Date'],
                         y = ticker_3_df['sma'] + (ticker_3_df['std'] * 2),
                         line_color = 'tan',
                         line = {'dash': 'dash'},
                         name = 'upper band',
                         opacity = 0.5),
              row = 1, col = 1) 
# Lower Bound fill in between with parameter 'fill': 'tonexty'
rsi_plot3=rsi_plot3.add_trace(go.Scatter(x = ticker_3_df['Date'],
                         y = ticker_3_df['sma'] - (ticker_3_df['std'] * 2),
                         line_color = 'tan',
                         line = {'dash': 'dash'},
                         fill = 'tonexty',
                         name = 'lower band',
                         opacity = 0.5),
              row = 1, col = 1)

# Labels & Titles
rsi_plot3=rsi_plot3.update_layout(
    height=500,
    width=1200,
    xaxis_title="Volume(Millions)",
    yaxis_title="Price",
    legend_title="Indicators",
    )
# ----------------
# Volume Plot
rsi_plot3=rsi_plot3.add_trace(go.Bar(x = ticker_3_df['Date'], y = ticker_3_df['volume'], showlegend=False), 
          row = 2, col = 1)
# Remove range slider; (short time frame)
rsi_plot3=rsi_plot3.update(layout_xaxis_rangeslider_visible=True)
#rsi_plot3.show()

In [36]:
#RSI PLOT 4 ticker_4_df

# Create subplots with 2 rows; top for candlestick price, and bottom for bar volume
rsi_plot4 = make_subplots(rows = 2, cols = 1, shared_xaxes = True, subplot_titles = (f'{ticker_4_df.symbol[0]} Relative Strength Index', None), vertical_spacing = 0.2, row_heights = [1.2, .8])

# Candlestick Plot
rsi_plot4=rsi_plot4.add_trace(go.Candlestick(x = ticker_4_df['Date'],
                             open = ticker_4_df['open'],
                             high = ticker_4_df['high'],
                             low = ticker_4_df['low'],
                             close = ticker_4_df['close'], showlegend=False,
                             increasing_line_color= 'olive', decreasing_line_color= 'darkred',
                             name = 'candlestick'),
              row = 1, col = 1)

# Moving Average
rsi_plot4=rsi_plot4.add_trace(go.Scatter(x = ticker_4_df['Date'],
                         y = ticker_4_df['sma'],
                         line_color = 'black',
                         name = 'sma'),
              row = 1, col = 1)

# Upper Bound
rsi_plot4=rsi_plot4.add_trace(go.Scatter(x = ticker_4_df['Date'],
                         y = ticker_4_df['sma'] + (ticker_4_df['std'] * 2),
                         line_color = 'tan',
                         line = {'dash': 'dash'},
                         name = 'upper band',
                         opacity = 0.5),
              row = 1, col = 1) 
# Lower Bound fill in between with parameter 'fill': 'tonexty'
rsi_plot4=rsi_plot4.add_trace(go.Scatter(x = ticker_4_df['Date'],
                         y = ticker_4_df['sma'] - (ticker_4_df['std'] * 2),
                         line_color = 'tan',
                         line = {'dash': 'dash'},
                         fill = 'tonexty',
                         name = 'lower band',
                         opacity = 0.5),
              row = 1, col = 1)

# Labels & Titles
rsi_plot4=rsi_plot4.update_layout(
    height=500,
    width=1200,
    xaxis_title="Volume(Millions)",
    yaxis_title="Price",
    legend_title="Indicators",
    )
# ----------------

# Volume Plot
rsi_plot4=rsi_plot4.add_trace(go.Bar(x = ticker_4_df['Date'], y = ticker_4_df['volume'], showlegend=False), 
          row = 2, col = 1)
# Remove range slider; (short time frame)
rsi_plot4=rsi_plot4.update(layout_xaxis_rangeslider_visible=True)
#rsi_plot4.show()

In [37]:
#RSI PLOT 5 ticker_5_df

# Create subplots with 2 rows; top for candlestick price, and bottom for bar volume
rsi_plot5 = make_subplots(rows = 2, cols = 1, shared_xaxes = True, subplot_titles = (f'{ticker_5_df.symbol[0]} Relative Strength Index', None ), vertical_spacing = 0.2, row_heights = [1.2, .8])

# Candlestick Plot
rsi_plot5=rsi_plot5.add_trace(go.Candlestick(x = ticker_5_df['Date'],
                             open = ticker_5_df['open'],
                             high = ticker_5_df['high'],
                             low = ticker_5_df['low'],
                             close = ticker_5_df['close'], showlegend=False,
                             increasing_line_color= 'olive', decreasing_line_color= 'darkred',
                             name = 'candlestick'),
              row = 1, col = 1)

# Moving Average
rsi_plot5=rsi_plot5.add_trace(go.Scatter(x = ticker_5_df['Date'],
                         y = ticker_5_df['sma'],
                         line_color = 'black',
                         name = 'sma'),
              row = 1, col = 1)

# Upper Bound
rsi_plot5=rsi_plot5.add_trace(go.Scatter(x = ticker_5_df['Date'],
                         y = ticker_5_df['sma'] + (ticker_5_df['std'] * 2),
                         line_color = 'tan',
                         line = {'dash': 'dash'},
                         name = 'upper band',
                         opacity = 0.5),
              row = 1, col = 1) 
# Lower Bound fill in between with parameter 'fill': 'tonexty'
rsi_plot5=rsi_plot5.add_trace(go.Scatter(x = ticker_5_df['Date'],
                         y = ticker_5_df['sma'] - (ticker_5_df['std'] * 2),
                         line_color = 'tan',
                         line = {'dash': 'dash'},
                         fill = 'tonexty',
                         name = 'lower band',
                         opacity = 0.5),
              row = 1, col = 1)

# Labels & Titles
rsi_plot5=rsi_plot5.update_layout(
    height=500,
    width=1200,
    xaxis_title="Volume(Millions)",
    yaxis_title="Price",
    legend_title="Indicators",
    )
# ----------------

# Volume Plot
rsi_plot5=rsi_plot5.add_trace(go.Bar(x = ticker_5_df['Date'], y = ticker_5_df['volume'], showlegend=False), 
          row = 2, col = 1)
# Remove range slider; (short time frame)
rsi_plot5=rsi_plot5.update(layout_xaxis_rangeslider_visible=True)
#rsi_5.show()

In [38]:

#df_list = [ticker_1_df, ticker_2_df, ticker_3_df, ticker_4_df, ticker_5_df]
candle_plot1= go.Figure(
data=[
    go.Candlestick(x=ticker_1_df['Date'],
                   open=ticker_1_df['open'],
                   high=ticker_1_df['high'],
                   low=ticker_1_df['low'],
                   close=ticker_1_df['close'],
                   increasing_line_color= 'olive', decreasing_line_color= 'darkred')
])    
candle_plot1=candle_plot1.update(layout_xaxis_rangeslider_visible=True)
candle_plot1=candle_plot1.update_layout(
    height=500,
    width=1200,
    title=f'{ticker_1_df.symbol[0]} Daily Price',
    xaxis_title='Date',
    yaxis_title='Price (USD)')
#candle_plot1.show()

#---------------------------------------------

#candle 2 ticker 2
candle_plot2= go.Figure(
data=[
    go.Candlestick(x=ticker_2_df['Date'],
                   open=ticker_2_df['open'],
                   high=ticker_2_df['high'],
                   low=ticker_2_df['low'],
                   close=ticker_2_df['close'],
                   increasing_line_color= 'olive', decreasing_line_color= 'darkred')
])    
candle_plot2=candle_plot2.update(layout_xaxis_rangeslider_visible=True)
candle_plot2=candle_plot2.update_layout(
    height=500,
    width=1200,
    title=f'{ticker_2_df.symbol[0]} Daily Price ',
    xaxis_title='Date',
    yaxis_title='Price (USD)')
#candle_plot2.show()

# #---------------------------------------------


#candle 3 ticker 3
candle_plot3= go.Figure(
data=[
    go.Candlestick(x=ticker_3_df['Date'],
                   open=ticker_3_df['open'],
                   high=ticker_3_df['high'],
                   low=ticker_3_df['low'],
                   close=ticker_3_df['close'],
                   increasing_line_color= 'olive', decreasing_line_color= 'darkred')
])    
candle_plot3=candle_plot3.update(layout_xaxis_rangeslider_visible=True)
candle_plot3=candle_plot3.update_layout(
    height=500,
    width=1200,
    title=f'{ticker_3_df.symbol[0]} Daily Price',
    xaxis_title='Date',
    yaxis_title='Price (USD)')
#candle_plot3.show()

# #---------------------------------------------


#candle 4 ticker 4
candle_plot4= go.Figure(
data=[
    go.Candlestick(x=ticker_4_df['Date'],
                   open=ticker_4_df['open'],
                   high=ticker_4_df['high'],
                   low=ticker_4_df['low'],
                   close=ticker_4_df['close'],
                   increasing_line_color= 'olive', decreasing_line_color= 'darkred')
])    
candle_plot4=candle_plot4.update(layout_xaxis_rangeslider_visible=True)
candle_plot4=candle_plot4.update_layout(
    height=500,
    width=1200,
    title=f'{ticker_4_df.symbol[0]} Daily Price',
    xaxis_title='Date',
    yaxis_title='Price (USD)')
#candle_plot4.show()

#---------------------------------------------


# #candle 5 ticker 5
candle_plot5= go.Figure(
data=[
    go.Candlestick(x=ticker_5_df['Date'],
                   open=ticker_5_df['open'],
                   high=ticker_5_df['high'],
                   low=ticker_5_df['low'],
                   close=ticker_5_df['close'],
                   increasing_line_color= 'olive', decreasing_line_color= 'darkred')
])    
candle_plot5=candle_plot5.update(layout_xaxis_rangeslider_visible=True)
candle_plot5=candle_plot5.update_layout(
    height=500,
    width=1200,
    title=f'{ticker_5_df.symbol[0]} Daily Price',
    xaxis_title='Date',
    yaxis_title='Price (USD)')
#candle_plot5.show()

In [39]:
monte_row = pn.Row(fig1)
monte_row2 = pn.Row(fig2)
sim_stats = pn.Column("## Cryptara's Simulated Crypto-Portfolio Performance", monte_row, monte_row2)
investment_sim = pn.Column("## Your Investment's Predicted Outcomes", fig3)
candlestick_row1 = pn.Row(candle_plot1,candle_plot2)
candlestick_row2 = pn.Row(candle_plot3,candle_plot4)
candlestick_row3 = pn.Row(candle_plot5)
candlestick_column = pn.Column("## Daily Candlestick Charts", candlestick_row1,candlestick_row2,candlestick_row3)
row_rsi = pn.Row(rsi_plot1, rsi_plot2)
row2_rsi = pn.Row(rsi_plot3, rsi_plot4)
row3_rsi = pn.Row(rsi_plot5)
column_rsi = pn.Column("## Relative Strength Index Charts: Cryptara's Picks", row_rsi,row2_rsi, row3_rsi)


dashboard = pn.Tabs(
    ("RSI CHART", column_rsi),
    ("Candlestick Charts",candlestick_column),
    ("Simulated Statistics", sim_stats),
    ("Investment Returns",investment_sim),
    
)
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'6ae15216-f5ec-4ca2-846f-7565d4d24472': {'defs': …

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)
        [2] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)
        [3] Row
            [0] Plotly(Figure)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)
        [2] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)
        [3] Row
            [0] Plotly(Figure)
    [2] Column
        [0] Markdown(str)
        [1] Row
            [0] Plotly(Figure)
        [2] Row
            [0] Plotly(Figure)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)

In [40]:
pn.serve(dashboard)

Launching server at http://localhost:54473
